In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import yfinance as yf
nasdaq=yf.download("^IXIC", start='2003-11-14')
data=nasdaq
import pandas as pd
data
df = data
df['Rate'] = 0
for i in range(len(df)):
    if i == 5036:
        break
    df['Rate'][i] = ((df['Close'][i+1] - df['Close'][i])/df['Close'][i])*100

[*********************100%%**********************]  1 of 1 completed


<ipython-input-25-e81c9dfd88c4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rate'][i] = ((df['Close'][i+1] - df['Close'][i])/df['Close'][i])*100


In [ ]:
for i in range(len(df['Rate'])):
    if df['Rate'][i] > 0.7:
        df['Rate'][i] = 3
    elif df['Rate'][i] > 0.097:
        df['Rate'][i] = 2
    elif df['Rate'][i] > -0.5:
        df['Rate'][i] = 1
    else:
        df['Rate'][i] = 0
Rate_data = df

In [ ]:
Rate_data["Rate"].value_counts()

0.0    1307
3.0    1283
2.0    1235
1.0    1212
Name: Rate, dtype: int64

In [ ]:
df[["Rate"]].max(),df[["Rate"]].min(),df[["Rate"]].mean(), df[["Rate"]].quantile(.75)

(Rate    3.0
 dtype: float64,
 Rate    0.0
 dtype: float64,
 Rate    1.495136
 dtype: float64,
 Rate    3.0
 Name: 0.75, dtype: float64)

In [ ]:
df[["Rate"]].quantile(.50),df[["Rate"]].quantile(.25)

(Rate    1.0
 Name: 0.5, dtype: float64,
 Rate    0.0
 Name: 0.25, dtype: float64)

In [ ]:
gold_data = yf.download('GC=F', start='2003-11-17')
usd_data = yf.download('USDKRW=X', start='2003-11-17')
bond_data = yf.download('^IRX', start='2003-11-17')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
gold_data = gold_data.rename(columns = {'Open':'gold_Open', 'High':'gold_high', 'Low':'gold_Low', 'Close':'gold_Close'})
usd_data = usd_data.rename(columns = {'Open':'usd_Open', 'High':'usd_high', 'Low':'usd_Low', 'Close':'usd_Close'})
bond_data = bond_data.rename(columns = {'Open':'bond_Open', 'High':'bond_high', 'Low':'bond_Low', 'Close':'bond_Close'})

In [ ]:
gold_data = gold_data.diff(1)
usd_data = usd_data.diff(1)
bond_data = bond_data.diff(1)
gold_data = gold_data.iloc[:, :4]
usd_data = usd_data.iloc[:, :4]
bond_data = bond_data.iloc[: ,:4]

In [ ]:
first = pd.concat([gold_data,usd_data],axis=1)
second=pd.concat([first,bond_data],axis=1)
second

,gold_Open,gold_high,gold_Low,gold_Close,usd_Open,usd_high,usd_Low,usd_Close,bond_Open,bond_high,bond_Low,bond_Close
Date,,,,,,,,,,,,
2003-11-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-11-18,0.000000,-0.399994,-0.399994,6.100006,NaN,NaN,NaN,NaN,0.010,0.006,0.004,0.008
2003-11-19,0.100006,0.100006,0.100006,-2.700012,NaN,NaN,NaN,NaN,-0.001,-0.006,0.000,-0.009
2003-11-20,-4.000000,-4.000000,-4.000000,-1.199982,NaN,NaN,NaN,NaN,-0.005,-0.005,-0.007,-0.005
2003-11-21,0.299988,2.099976,0.299988,2.299988,NaN,NaN,NaN,NaN,-0.004,-0.004,-0.013,-0.010
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14,25.900024,18.600098,27.800049,16.300049,-0.550049,4.079956,-16.229980,-0.550049,0.032,0.020,0.000,-0.007
2023-11-15,-3.000000,-3.000000,-1.800049,-1.700073,-14.829956,-21.900024,-9.699951,-14.829956,-0.030,-0.022,-0.023,0.002
2023-11-16,2.500000,21.099976,4.599976,23.800049,-2.380005,1.520020,-6.030029,-2.380005,-0.002,-0.010,0.005,-0.012


In [ ]:
Rate_y_data = Rate_data.iloc[:,6:]
data = pd.concat([second,Rate_y_data],axis=1)
data = data.dropna(axis=0)
x_select = data.iloc[:, :]
y_select = data['Rate']

In [ ]:
x_select.shape, y_select.shape

((4978, 13), (4978,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_select, y_select, test_size = 0.33, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3335, 13) (1643, 13) (3335,) (1643,)


In [ ]:
n_input = x_train.shape[1]

n_output = len(list(set(y_train)))

print(f'n_input: {n_input} n_output: {n_output}')

n_input: 13 n_output: 4


In [ ]:
!pip install torchinfo
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4133 sha256=2a623c25ee3c849ed701903b21bb0bbe08a45996764e54566f5ca014b7e4f5bc
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot

class Net(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.l1 = nn.Linear(n_input, n_output)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x1 = self.l1(x)
        #x2 = self.dropout(x1)
        return x1

net = Net(n_input, n_output)

In [ ]:
for parameter in net.named_parameters():
  print(parameter)

('l1.weight', Parameter containing:
tensor([[-2.7254e-04, -1.0738e-01,  4.2490e-02, -1.7798e-02,  6.7200e-02,
         -1.1320e-01,  1.3320e-01, -2.2187e-01,  1.3088e-01, -2.0725e-02,
         -2.3627e-01,  1.4685e-01, -1.5636e-01],
        [ 2.3892e-01, -8.8058e-02, -1.8598e-01, -1.8033e-02,  1.9964e-01,
          1.7016e-01, -1.6481e-02,  1.4939e-01,  2.1163e-01, -1.6137e-01,
          2.3077e-01, -1.7575e-01, -2.4235e-01],
        [ 1.2050e-01, -2.2825e-01, -2.4486e-01, -1.5203e-01, -7.8342e-02,
          2.6731e-02, -9.5012e-02,  1.9147e-01,  6.3654e-03, -2.2416e-01,
         -2.7483e-02,  1.4004e-01, -1.3073e-01],
        [ 1.8248e-01, -9.2453e-02,  2.3595e-01,  1.3644e-01,  1.8347e-01,
          2.7705e-01, -1.2191e-01, -2.3751e-01, -4.8772e-02,  2.2364e-02,
          1.2219e-01,  1.7013e-01,  1.6574e-01]], requires_grad=True))
('l1.bias', Parameter containing:
tensor([-0.1567, -0.1108,  0.0407,  0.1155], requires_grad=True))


In [ ]:
criterion = nn.CrossEntropyLoss()

lr = 0.01

optimizer = optim.Adam(net.parameters(), lr=lr)

In [ ]:
inputs = torch.tensor(x_train.values).float()
labels = torch.tensor(y_train).long()

inputs_test = torch.tensor(x_test.values).float()
labels_test = torch.tensor(y_test).long()

In [ ]:
import numpy as np

lr = 0.01

net = Net(n_input, n_output)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)

num_epochs = 10000

history = np.zeros((0,5))

In [ ]:
labels_test

tensor([2, 0, 0,  ..., 2, 3, 3])

In [ ]:
# 반복 계산 메인 루프

for epoch in range(num_epochs):

    # 훈련 페이즈

    # 경사 초기화
    optimizer.zero_grad()

    # 예측 계산
    outputs = net(inputs)

    # 손실 계산
    loss = criterion(outputs, labels)

    # 경사 계산
    loss.backward()

    # 파라미터 수정
    optimizer.step()

    # 예측 라벨 산출
    predicted = torch.max(outputs, 1)[1]

    # 손실과 정확도 계산
    train_loss = loss.item()
    train_acc = (predicted == labels).sum()  / len(labels)

    # 예측 페이즈

    # 예측 계산
    outputs_test = net(inputs_test)

    # 손실 계산
    loss_test = criterion(outputs_test, labels_test)

    # 예측 라벨 산출
    predicted_test = torch.max(outputs_test, 1)[1]

    # 손실과 정확도 계산
    val_loss =  loss_test.item()
    val_acc =  (predicted_test == labels_test).sum() / len(labels_test)

    if ((epoch) % 10 == 0):
        print (f'Epoch [{epoch}/{num_epochs}], loss: {train_loss:.5f} acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}')
        item = np.array([epoch, train_loss, train_acc, val_loss, val_acc])
        history = np.vstack((history, item))

Epoch [0/10000], loss: 5.42788 acc: 0.24138 val_loss: 8.70980, val_acc: 0.24041
Epoch [10/10000], loss: 2.91513 acc: 0.24948 val_loss: 6.83350, val_acc: 0.24346
Epoch [20/10000], loss: 1.94675 acc: 0.24378 val_loss: 4.33911, val_acc: 0.23554
Epoch [30/10000], loss: 1.53545 acc: 0.28156 val_loss: 2.36204, val_acc: 0.31467
Epoch [40/10000], loss: 1.32765 acc: 0.35922 val_loss: 1.47753, val_acc: 0.36884
Epoch [50/10000], loss: 1.27009 acc: 0.38681 val_loss: 1.25324, val_acc: 0.39014
Epoch [60/10000], loss: 1.19856 acc: 0.42879 val_loss: 1.18533, val_acc: 0.43579
Epoch [70/10000], loss: 1.14441 acc: 0.49595 val_loss: 1.20356, val_acc: 0.49787
Epoch [80/10000], loss: 1.10552 acc: 0.55802 val_loss: 1.23788, val_acc: 0.55265
Epoch [90/10000], loss: 1.07676 acc: 0.56732 val_loss: 1.07455, val_acc: 0.56299
Epoch [100/10000], loss: 1.04261 acc: 0.60900 val_loss: 1.04169, val_acc: 0.60256
Epoch [110/10000], loss: 1.00994 acc: 0.65727 val_loss: 1.06074, val_acc: 0.64394
Epoch [120/10000], loss: 0.

In [ ]:
import numpy as np
acc=0
result=net(inputs_test)
for i,j in enumerate(result):
  if np.argmax(j.detach().numpy())==int(labels_test[i]):
    acc+=1
print(acc/len(labels_test)*100)

99.93913572732805


In [ ]:
import yfinance as yf
nasdaq=yf.download("^IXIC", start='2005-06-03')
data=nasdaq
import pandas as pd
data
df = data
df['Rate'] = 0
for i in range(len(df)):
    if i == len(df)-1:
        break
    df['Rate'][i] = ((df['Close'][i+1] - df['Close'][i])/df['Close'][i])*100

[*********************100%%**********************]  1 of 1 completed


<ipython-input-46-9903b34647f4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rate'][i] = ((df['Close'][i+1] - df['Close'][i])/df['Close'][i])*100


In [ ]:
for i in range(len(df['Rate'])):
    if df['Rate'][i] > 0.7:
        df['Rate'][i] = 3
    elif df['Rate'][i] > 0.097:
        df['Rate'][i] = 2
    elif df['Rate'][i] > -0.5:
        df['Rate'][i] = 1
    else:
        df['Rate'][i] = 0
Rate_data = df

In [ ]:
Rate_data["Rate"].value_counts()

0.0    1199
3.0    1188
2.0    1135
1.0    1126
Name: Rate, dtype: int64

In [ ]:
len(df)

4648

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
#import FinanceDataReader as fdr
import yfinance as yf
from torch.utils.data import Dataset, DataLoader

# 주식 데이터 가져오기
def get_stock_data(stock_code, start_date):
    df = yf.download(stock_code, start_date)
    mean=df.mean()
    std=df.std()
    df=(df-df.mean())/std
    df['sma5']=df.Close.rolling(5).mean() #1주
    df['sma20']=df.Close.rolling(20).mean() #4주
    df['sma50']=df.Close.rolling(50).mean() #한 분기
    df['sma100']=df.Close.rolling(100).mean() #반년
    df['sma200']=df.Close.rolling(200).mean() #1년
    return df[['Open','High','Low','Volume','Close','sma5','sma20','sma50','sma100','sma200']][199:]

from sklearn.model_selection import train_test_split
# 데이터 전처리
def preprocess_data(data, seq_length):
    sequences = []
    labels = []

    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = 1 if data[i+seq_length,4] > data[i+seq_length-1,4] else 0

        sequences.append(seq)
        labels.append(label)
    sequences=np.array(sequences)
    labels=np.array(labels)

    return sequences,labels

# LSTM 모델 정의
class StockPredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockPredictionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # 최종 타임 스텝의 출력만 사용
        output = self.fc(lstm_out)
        return output

# 주식 데이터 로딩 및 전처리
stock_code = 'GOOG'
start_date = '2004-06-01'
end_date = '2022-01-01'
seq_length = 20

data = get_stock_data(stock_code, start_date)
stock_data=data[:"2020-01-01"]
sequences,labels = preprocess_data(stock_data.values, seq_length)

# 데이터를 PyTorch 텐서로 변환
sequences = torch.FloatTensor(sequences)
labels = torch.LongTensor(labels)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# 데이터셋 및 데이터로더 생성
class StockDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

dataset = StockDataset(sequences, labels)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델, 손실 함수, 최적화 함수 초기화
input_size = 10
hidden_size = 32
output_size = 2  # 0 또는 1을 출력하기 위한 이진 분류
model = StockPredictionModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0000001)

In [ ]:
# 훈련
num_epochs = 100
for epoch in range(num_epochs):
    for seq, label in data_loader:
        optimizer.zero_grad()
        output = model(seq)
        loss = criterion(output, label.squeeze())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/100], Loss: 0.7165123224258423
Epoch [2/100], Loss: 0.6470819711685181
Epoch [3/100], Loss: 0.712046205997467
Epoch [4/100], Loss: 0.6498315334320068
Epoch [5/100], Loss: 0.6944506168365479
Epoch [6/100], Loss: 0.7616297602653503
Epoch [7/100], Loss: 0.6331384778022766
Epoch [8/100], Loss: 0.7382029294967651
Epoch [9/100], Loss: 0.6778653860092163
Epoch [10/100], Loss: 0.7390950322151184
Epoch [11/100], Loss: 0.6582087874412537
Epoch [12/100], Loss: 0.6887161731719971
Epoch [13/100], Loss: 0.7487218976020813
Epoch [14/100], Loss: 0.6424516439437866
Epoch [15/100], Loss: 0.691466748714447
Epoch [16/100], Loss: 0.6839209794998169
Epoch [17/100], Loss: 0.6947861313819885
Epoch [18/100], Loss: 0.7223840355873108
Epoch [19/100], Loss: 0.6430363655090332
Epoch [20/100], Loss: 0.7614601850509644
Epoch [21/100], Loss: 0.6828271150588989
Epoch [22/100], Loss: 0.6451970338821411
Epoch [23/100], Loss: 0.7343126535415649
Epoch [24/100], Loss: 0.6966772079467773
Epoch [25/100], Loss: 0.735

In [ ]:
# 테스트
model.eval()
with torch.no_grad():
    test_sequence = torch.FloatTensor(sequences).view(-1,seq_length,10)
    predicted_prob = model(test_sequence)
    predicted_class = torch.argmax(predicted_prob, dim=1)

In [ ]:
import numpy as np
x=np.linspace(10,10000,4000)
(x-x.mean())/x.std()

array([-1.73161785, -1.73075182, -1.7298858 , ...,  1.7298858 ,
        1.73075182,  1.73161785])

In [ ]:
predicted_class[0]
y_test[0]
res=[int(predicted_class[i].numpy())==int(labels[i]) for i in range(1528)]
pd.Series(res).value_counts()

True     775
False    753
dtype: int64

In [ ]:
(773)/1528*100

50.589005235602095

In [ ]:
def predict_next_price(model, test_sequence):
    model.eval()
    with torch.no_grad():
      if test_sequence.shape==(10,):
        test_sequence = torch.FloatTensor(test_sequence).view(1, 1, 10)
        predicted_prob = model(test_sequence)
        predicted_class = torch.argmax(predicted_prob, dim=1).item()
      else:
        test_sequence = torch.FloatTensor(test_sequence).view(1, len(test_sequence), len(test_sequence[0]))
        predicted_prob = model(test_sequence)
        predicted_class = torch.argmax(predicted_prob, dim=1).item()
    return predicted_class

In [ ]:
stock_data.values[:1,:].shape

(1, 10)

In [ ]:
result=[]
for i in range(len(stock_data.values)):
  if i<20:
    result.append(predict_next_price(model,stock_data.values[:(i+1),:]))
  else:
    result.append(predict_next_price(model,stock_data.values[i-19:(i+1),:]))

In [ ]:
stock_data.reset_index(drop=False,inplace=True)
df_concat=pd.concat([stock_data,pd.Series(result)],axis=1)

In [ ]:
df_concat[0].value_counts()

1    3670
Name: 0, dtype: int64

In [ ]:
df_concat

,Date,Open,High,Low,Volume,Close,sma5,sma20,sma50,sma100,sma200,0
0,2005-06-03,-0.889233,-0.888828,-0.893615,4.195643,-0.893279,-0.893522,-0.913358,-0.932695,-0.942402,-0.958669,0
1,2005-06-06,-0.892090,-0.885972,-0.890716,5.188807,-0.886352,-0.890286,-0.911318,-0.931246,-0.941770,-0.958051,0
2,2005-06-07,-0.882541,-0.882224,-0.885160,5.665572,-0.884938,-0.888230,-0.909141,-0.929797,-0.941136,-0.957452,0
3,2005-06-08,-0.885300,-0.886331,-0.893228,6.031124,-0.893733,-0.889325,-0.907463,-0.928500,-0.940590,-0.956900,0
4,2005-06-09,-0.890577,-0.889341,-0.891549,3.574531,-0.889355,-0.889531,-0.905678,-0.927126,-0.940030,-0.956311,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4643,2023-11-13,2.400290,2.381118,2.420884,-0.678807,2.405416,2.389008,2.364775,2.429345,2.330259,2.028445,0
4644,2023-11-14,2.459973,2.461768,2.482244,-0.639773,2.452034,2.404791,2.357534,2.428678,2.333491,2.033325,0
4645,2023-11-15,2.485775,2.451461,2.488300,-0.682566,2.476776,2.421042,2.353758,2.429204,2.337994,2.038080,0
4646,2023-11-16,2.494115,2.504026,2.508051,-0.670843,2.537197,2.457555,2.353393,2.430506,2.343122,2.042933,0


In [ ]:
df_concat=df_concat.set_index("Date")

In [ ]:
#df_concat.reset_index(drop=False,inplace=True)
#df.reset_index(drop=True,inplace=True)
df_final=pd.concat([df_concat,df],axis=1)
df_final

,Date,Open,High,Low,Volume,Close,sma5,sma20,sma50,sma100,...,0,index,Date,Open,High,Low,Close,Adj Close,Volume,Rate
0,2005-06-03,-0.889233,-0.888828,-0.893615,4.195643,-0.893279,-0.893522,-0.913358,-0.932695,-0.942402,...,0,0,2005-06-03,2094.179932,2095.989990,2069.169922,2071.429932,2071.429932,1637900000,2.0
1,2005-06-06,-0.892090,-0.885972,-0.890716,5.188807,-0.886352,-0.890286,-0.911318,-0.931246,-0.941770,...,0,1,2005-06-06,2072.360107,2078.610107,2066.360107,2075.760010,2075.760010,1477150000,1.0
2,2005-06-07,-0.882541,-0.882224,-0.885160,5.665572,-0.884938,-0.888230,-0.909141,-0.929797,-0.941136,...,0,2,2005-06-07,2079.060059,2095.959961,2067.139893,2067.159912,2067.159912,1846530000,1.0
3,2005-06-08,-0.885300,-0.886331,-0.893228,6.031124,-0.893733,-0.889325,-0.907463,-0.928500,-0.940590,...,0,3,2005-06-08,2073.209961,2074.610107,2057.580078,2060.179932,2060.179932,1602540000,3.0
4,2005-06-09,-0.890577,-0.889341,-0.891549,3.574531,-0.889355,-0.889531,-0.905678,-0.927126,-0.940030,...,0,4,2005-06-09,2059.580078,2077.469971,2052.959961,2076.909912,2076.909912,1666670000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,2023-11-13,2.400290,2.381118,2.420884,-0.678807,2.405416,2.389008,2.364775,2.429345,2.330259,...,0,4643,2023-11-13,13745.959961,13805.110352,13687.780273,13767.740234,13767.740234,4349590000,3.0
4644,2023-11-14,2.459973,2.461768,2.482244,-0.639773,2.452034,2.404791,2.357534,2.428678,2.333491,...,0,4644,2023-11-14,14015.370117,14124.059570,14003.139648,14094.379883,14094.379883,5332490000,1.0
4645,2023-11-15,2.485775,2.451461,2.488300,-0.682566,2.476776,2.421042,2.353758,2.429204,2.337994,...,0,4645,2023-11-15,14147.269531,14194.360352,14060.700195,14103.839844,14103.839844,5063830000,1.0
4646,2023-11-16,2.494115,2.504026,2.508051,-0.670843,2.537197,2.457555,2.353393,2.430506,2.343122,...,0,4646,2023-11-16,14066.879883,14130.450195,14033.790039,14113.669922,14113.669922,4545180000,1.0


In [ ]:
df_final=df_final.set_index(df_final.iloc[:,0])
df_final

,Date,Open,High,Low,Volume,Close,sma5,sma20,sma50,sma100,...,0,index,Date,Open,High,Low,Close,Adj Close,Volume,Rate
Date,,,,,,,,,,,,,,,,,,,,,
2005-06-03,2005-06-03,-0.889233,-0.888828,-0.893615,4.195643,-0.893279,-0.893522,-0.913358,-0.932695,-0.942402,...,0,0,2005-06-03,2094.179932,2095.989990,2069.169922,2071.429932,2071.429932,1637900000,2.0
2005-06-06,2005-06-06,-0.892090,-0.885972,-0.890716,5.188807,-0.886352,-0.890286,-0.911318,-0.931246,-0.941770,...,0,1,2005-06-06,2072.360107,2078.610107,2066.360107,2075.760010,2075.760010,1477150000,1.0
2005-06-07,2005-06-07,-0.882541,-0.882224,-0.885160,5.665572,-0.884938,-0.888230,-0.909141,-0.929797,-0.941136,...,0,2,2005-06-07,2079.060059,2095.959961,2067.139893,2067.159912,2067.159912,1846530000,1.0
2005-06-08,2005-06-08,-0.885300,-0.886331,-0.893228,6.031124,-0.893733,-0.889325,-0.907463,-0.928500,-0.940590,...,0,3,2005-06-08,2073.209961,2074.610107,2057.580078,2060.179932,2060.179932,1602540000,3.0
2005-06-09,2005-06-09,-0.890577,-0.889341,-0.891549,3.574531,-0.889355,-0.889531,-0.905678,-0.927126,-0.940030,...,0,4,2005-06-09,2059.580078,2077.469971,2052.959961,2076.909912,2076.909912,1666670000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13,2023-11-13,2.400290,2.381118,2.420884,-0.678807,2.405416,2.389008,2.364775,2.429345,2.330259,...,0,4643,2023-11-13,13745.959961,13805.110352,13687.780273,13767.740234,13767.740234,4349590000,3.0
2023-11-14,2023-11-14,2.459973,2.461768,2.482244,-0.639773,2.452034,2.404791,2.357534,2.428678,2.333491,...,0,4644,2023-11-14,14015.370117,14124.059570,14003.139648,14094.379883,14094.379883,5332490000,1.0
2023-11-15,2023-11-15,2.485775,2.451461,2.488300,-0.682566,2.476776,2.421042,2.353758,2.429204,2.337994,...,0,4645,2023-11-15,14147.269531,14194.360352,14060.700195,14103.839844,14103.839844,5063830000,1.0


In [ ]:
df_final_real=df_final.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,20]]

In [ ]:
a=df_final_real["2015-01-01":]


In [ ]:
#!pip install backtesting
from backtesting import Backtest,Strategy
from backtesting.lib import crossover

class backtesting(Strategy):
  def init(self):
    super().init()
  def next(self):
    for i in range(len(self.data)):
      try:
        if self.data[0][i]==1 and self.data["Rate"][i]>=2.0:
            self.buy()
        else:
            self.sell()
      except:
        pass

In [ ]:
bt=Backtest(df_final_real["2010-01-01":],backtesting,commission=.0002,cash=100000,exclusive_orders=True)
stats=bt.run()
print(stats)

Start                     2010-01-04 00:00:00
End                       2023-11-17 00:00:00
Duration                   5065 days 00:00:00
Exposure Time [%]                   51.316543
Equity Final [$]                          0.0
Equity Peak [$]           28641872841073.1...
Return [%]                             -100.0
Buy & Hold Return [%]             -472.665711
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]             1478.172716
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                   -5.091955
Max. Drawdown Duration     2480 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                 1793
Win Rate [%]                        61.851645
Best Trade [%]                     211.886188
Worst Trade [%]                   -248.410685
Avg. Trade [%]                    

In [ ]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2556', ...)

In [ ]:
#stock_data.reset_index(drop=False,inplace=True)
#df.reset_index(drop=False,inplace=True)
compare=pd.concat([stock_data,df],axis=1)

In [ ]:
final_compare=pd.concat([compare,pd.Series(result)],axis=1)
final_compare[final_compare["Rate"]==2.0][0].describe()

count    1135.000000
mean        0.999119
std         0.029683
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: 0, dtype: float64

In [ ]:
final_compare[final_compare["Rate"]==2.0]["Y"].describe()

count    1135.000000
mean        0.635242
std         0.481574
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Y, dtype: float64

In [ ]:
final_compare[final_compare["Rate"]==1.0][0].describe()

count    1126.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: 0, dtype: float64

In [ ]:
final_compare[final_compare["Rate"]==1.0]["Y"].describe()

count    1126.000000
mean        0.424512
std         0.494488
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: Y, dtype: float64

In [ ]:
final_compare[final_compare["Rate"]==3.0][0].describe()

count    1188.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: 0, dtype: float64

In [ ]:
final_compare[final_compare["Rate"]==3.0]["Y"].describe()

count    1188.000000
mean        0.898148
std         0.302581
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Y, dtype: float64

In [8]:
'